In [1]:
!pip install surprise
!pip install scikit-surprise

In [3]:
%%time
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, coo_matrix
from tqdm.auto import tqdm

In [4]:
%%script false --no-raise-error

list_coord = []
idx = 0
test_tracks = {}

with open('/Users/david/Documents/GitHub/YandexCup_RecSys/train') as f:
    lines = f.readlines()
    for line in lines:
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
first_id_test = idx

with open('/Users/david/Documents/GitHub/YandexCup_RecSys/test') as f:
    lines = f.readlines()
    for line in lines:
        tracks = line.strip().split(' ')
        tracks_arr = np.array(tracks).astype(np.int32)
        test_tracks[idx] = tracks_arr
        for track in tracks:
            list_coord.append((idx, track))
        idx += 1
        
last_id_test = idx - 1

In [5]:
%%time

list_coord = []
idx = 0

with open('/Users/david/Documents/GitHub/YandexCup_RecSys/train') as f:
    lines = f.readlines()
    for line in lines:
        tracks = line.strip().split(' ')
        for track in tracks:
            list_coord.append( np.array([idx, int(track), 1]))
        idx += 1
        if idx > 10000:
            break


CPU times: user 568 ms, sys: 197 ms, total: 765 ms
Wall time: 904 ms


In [6]:
%%time
dataset = pd.DataFrame(np.array(list_coord), columns=['uid', 'iid', 'rating'])

CPU times: user 108 ms, sys: 18.5 ms, total: 126 ms
Wall time: 126 ms


In [7]:
from surprise import Dataset, Reader

In [30]:
%%time
from surprise import Dataset, Reader
reader = Reader(rating_scale=(0, 1))
data = Dataset.load_from_df(dataset, reader)

CPU times: user 285 ms, sys: 93.5 ms, total: 378 ms
Wall time: 459 ms


In [31]:
%%time
from surprise.model_selection import train_test_split
trainset, testset = train_test_split(data, test_size=.15)

CPU times: user 1.15 s, sys: 421 ms, total: 1.57 s
Wall time: 2.02 s


In [32]:
%%time
from surprise import KNNWithMeans, KNNBasic, accuracy

sim_options = {
    "name": "cosine",
    "user_based": True
}
algo = KNNWithMeans(k=40, min_k=1, sim_options=sim_options, verbose=True)
algo.fit(trainset)

Computing the cosine similarity matrix...
Done computing similarity matrix.
CPU times: user 2.59 s, sys: 15.1 s, total: 17.7 s
Wall time: 26.8 s


In [33]:
test_pred = algo.test(testset)
accuracy.rmse(test_pred, verbose=True)

RMSE: 0.0000


0.0

In [45]:
arr = dataset.iid.values

In [50]:
tot = 0
for i in arr:
    tot += algo.predict(uid=1, iid=i).est
tot / len(arr)

1.0

In [58]:
algo.predict(uid=0, iid=339459)

Prediction(uid=0, iid=339459, r_ui=None, est=1, details={'actual_k': 0, 'was_impossible': False})

In [56]:
dataset

,uid,iid,rating
0,0,333396,1
1,0,267089,1
2,0,155959,1
3,0,353335,1
4,0,414000,1
...,...,...,...
813904,10000,291151,1
813905,10000,462315,1
813906,10000,339459,1
813907,10000,133453,1
